In [1]:
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets)=boston_housing.load_data()

57026/57026 [==============================] - 0s 0us/step


In [3]:
train_data.shape

(404, 13)

In [4]:
test_data.shape

(102, 13)

In [5]:
train_targets

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

상이한 스케일을 가진 값을 신경망에 주입하면 문제가 됨. 네트워크가 이런 다양한 데이터에 자동으로 맞추려고 할 수 있지만 이는 확실히 학습을 더 어렵게 만듬.
상이한 스케일을 가진 갑을 다룰 때는 주로 특성별로 정규화를 함-입력 데이터에 있는 각 특성(입력 데이터 행렬의 열)에 대해서 특성의 평균을 빼고 표준편차로 나눔. 이렇게 하면 특성의 중앙이 0 근처에 맞춰지고 표준편차가 1이 됨(numpy를 통해서 간단히 할 수 있는 과정임)

In [6]:
#데이터 정규화-정규화할 때 사용하는 겂은 훈련데이터에서 사용한 값이어야 함(머신러닝 작업과정에서 절대로 테스트 데이터에서 계산한 어떤 값도 사용해서는 안됨!!!)
mean=train_data.mean(axis=0)
train_data -= mean
std=train_data.std(axis=0)
train_data /= std

test_data -= mean
test_data /= std

In [7]:
#모델 구성-샘플 개수가 적으므로 64개의 유닛을 가진 2개의 은닉 층으로 작은 네트워크를 구성하여 사용
#훈련데이터의 개수가 적을수록 과대적합이 더 쉽게 일어나므로 작은 모델을 사용하는 것이 과대적합을 피하는 한 방법(?)
from keras import models
from keras import layers

#동일한 모델을 여러 번 생성할 것이므로 함수를 만들어서 사용
def build_model():
    model=models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    #마지막 층은 하나의 유닛을 가지고 있고 활성화 함수가 없음(선형층이라고 부름)
    #이것이 전형적인 스칼라 회귀(하나의 연속적인 값을 예측하는 회귀)를 위한 구성
    #마지막 층에서 활성화함수를 적용하면 출력 값의 범위를 제한하게 됨
    #->예를 들어, sigmoid 활성화 함수를 적용하면 네트워크가 0과 1사이의 값을 예측하도록 학습될 것
    #->여기서는 마지막 층이 순수한 선형이므로 네트워크가 어떤 범위의 값이라도 예측하도록 자유롭게 학습됨
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])#훈련하는 동안 모니터링을 위해 MAE를 측정
    return model


In [13]:
#k-겹 검증을 사용한 훈련 검증
import numpy as np
k=4
num_val_samples=len(train_data)//k
num_epochs=100
all_scores=[]
for i  in range(k):
    print('처리 중인 폴드 #', i)
    val_data=train_data[i*num_val_samples: (i+1)*num_val_samples] #검증 데이터 준비: k번째 분할
    val_targets=train_targets[i*num_val_samples:(i+1)*num_val_samples]
    
    partial_train_data=np.concatenate( #훈련 데이터 준비: 다른 분할 전체
        [train_data[:i*num_val_samples],
         train_data[(i+1)*num_val_samples:]],
        axis=0
    )
    partial_train_targets=np.concatenate(
        [train_targets[:i*num_val_samples],
         train_targets[(i+1)*num_val_samples:]],
        axis=0
    )
    
model=build_model() #케라스 모델 구성(컴파일 포함)
model.fit(partial_train_data, partial_train_targets,#훈련모델(verbose=0이므로 훈련 과정이 출력되지 않음)
          epochs=num_epochs, batch_size=1, verbose=0)
val_mse, val_mae=model.evaluate(val_data, val_targets, verbose=0)#검증세트로 모델 
#verbose는 상세한 logging을 출력할지 말지를 조정하는 parameter
all_scores.append(val_mae)

처리 중인 폴드 # 0
처리 중인 폴드 # 1
처리 중인 폴드 # 2
처리 중인 폴드 # 3


In [14]:
all_scores #k=4인데 왜 예제와 달리 score가 하나 밖에 안 나오는가???

[2.6169018745422363]

In [12]:
np.mean(all_scores) #score가 하나 밖에 안 나오니 평균도 그 값 그대로 나옴;;;;;

2.6636569499969482

In [15]:
num_epochs = 500
all_mae_histories = []
for i in range(k):
    print('처리중인 폴드 #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]  #검증 데이터 준비: k번째 분할    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    partial_train_data = np.concatenate(  # 훈련 데이터 준비: 다른 분할 전체
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [train_targets[:i * num_val_samples],
         train_targets[(i + 1) * num_val_samples:]],
        axis=0)

model = build_model()  # 케라스 모델 구성(컴파일 포함)
history = model.fit(partial_train_data, partial_train_targets,  # 모델 훈련(verbose=0이므로 훈련 과정이 출력되지 않습니다.)
                    validation_data=(val_data, val_targets),
                    epochs=num_epochs, batch_size=1, verbose=0)
mae_history = history.history['val_mean_absolute_error']
all_mae_histories.append(mae_history)

처리중인 폴드 # 0
처리중인 폴드 # 1
처리중인 폴드 # 2
처리중인 폴드 # 3


KeyError: 'val_mean_absolute_error'

에러로 더 이상 진행 안 함